Below is a simple example of a job shop problem, in which each task is labeled by a pair of numbers (m, p) where m is the number of the machine the task must be processed on and p is the processing time of the task — the amount of time it requires. (The numbering of jobs and machines starts at 0.)

job 0 = [(0, 3), (1, 2), (2, 2)]

job 1 = [(0, 2), (2, 1), (1, 4)]

job 2 = [(1, 4), (2, 3)]

In the example, job 0 has three tasks. The first, (0, 3), must be processed on machine 0 in 3 units of time. The second, (1, 2), must be processed on machine 1 in 2 units of time, and so on. Altogether, there are eight tasks.


https://developers.google.com/optimization/scheduling/job_shop

In [1]:
import pandas as pd
import numpy as np
from docplex.cp.model import  CpoModel

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
def interval_var_series(df, mdl,**kargs):
    return pd.Series(mdl.interval_var_list(len(df), **kargs), index = df.index)

In [22]:
mdl = CpoModel(name="JobScheduling")

In [23]:
job_0 = [(0, 3), (1, 2), (2, 2)]
job_1 = [(0, 2), (2, 1), (1, 4)]
job_2 = [(1, 4), (2, 3)]

jobs_data = pd.DataFrame([])
i=0
for j in [job_0,  job_1,  job_2  ]: 
    u = pd.DataFrame(j, columns = ['machine','time'])
    u['job'] = i
    u['order'] = np.arange(0,len(u))
    jobs_data = pd.concat([jobs_data,u], axis = 0)
    i=i+1

jobs_data = jobs_data[['job','order', 'machine', 'time']].reset_index(drop = True)
jobs_data

,job,order,machine,time
0,0,0,0,3
1,0,1,1,2
2,0,2,2,2
3,1,0,0,2
4,1,1,2,1
5,1,2,1,4
6,2,0,1,4
7,2,1,2,3


In [24]:
horizon = jobs_data['time'].sum()
horizon

21

In [25]:
DV=[]
for row in jobs_data.itertuples():
    #print(i)
    dv = mdl.interval_var(size  =  row.time )# start = 0,end =horizon,  , name='assinged_%s_%s'%(row.job, row.machine)
    DV.append(dv) 
jobs_data['DV'] = DV
jobs_data

,job,order,machine,time,DV
0,0,0,0,3,intervalVar(size=3)
1,0,1,1,2,intervalVar(size=2)
2,0,2,2,2,intervalVar(size=2)
3,1,0,0,2,intervalVar(size=2)
4,1,1,2,1,intervalVar(size=1)
5,1,2,1,4,intervalVar(size=4)
6,2,0,1,4,intervalVar(size=4)
7,2,1,2,3,intervalVar(size=3)


mdl.add( mdl.end_before_start(DV[0], DV[1]) )
mdl.add( mdl.end_before_start(DV[0], DV[2]) )
mdl.add( mdl.end_before_start(DV[1], DV[2]) )


print("\nSolving model....")
msol = mdl.solve(TimeLimit=10)
print("done")

In [26]:
dv

In [27]:
all_jobs = jobs_data['job'].unique()
all_machines = jobs_data['machine'].unique()


def extract_precedence(m):
    AllOrders = []
    for i in m.keys():
        for j in m.keys():
            if m[i]<m[j]:
                AllOrders.append([i,j])

    return AllOrders


for i in all_jobs:
    u= jobs_data[jobs_data['job']==i]
    m = dict(zip(u.index, u.order.values))
    precedence_list = extract_precedence(m)

    for j in precedence_list:
        print("The task ", jobs_data.loc[j[0]]['DV'] , '   should be before   ', jobs_data.loc[j[1]]['DV'])

        mdl.add( mdl.end_before_start(jobs_data.loc[j[0]]['DV'], jobs_data.loc[j[1]]['DV']) )



The task  intervalVar(size=3)    should be before    intervalVar(size=2)
The task  intervalVar(size=3)    should be before    intervalVar(size=2)
The task  intervalVar(size=2)    should be before    intervalVar(size=2)
The task  intervalVar(size=2)    should be before    intervalVar(size=1)
The task  intervalVar(size=2)    should be before    intervalVar(size=4)
The task  intervalVar(size=1)    should be before    intervalVar(size=4)
The task  intervalVar(size=4)    should be before    intervalVar(size=3)


In [28]:
print("\nSolving model....")
msol = mdl.solve(TimeLimit=10)
print("done")


Solving model....
 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Satisfiability problem - 8 variables, 7 constraints
 ! TimeLimit            = 10
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 !  . Log search space  : 24.0 (before), 24.0 (after)
 !  . Memory usage      : 302.5 kB (before), 302.5 kB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed    W       Branch decision
 *                      6  0.01s        1            -
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 36
 ! Number of fails        : 0
 ! Total memory usage     : 2.5 MB (2.5 MB CP Optimizer + 0.0 MB Concert)
 ! Time spent in solve    : 0.02s (0.01s engine + 0.01s 

In [29]:
var_sol = msol.get_var_solution(DV[2])


In [81]:
DV[1]